In [1]:
import pandas as pd
import numpy as np
import pyodbc
import sys
import sqlalchemy as sql
import re
from tqdm import tqdm

sys.path.append(r'..\2023-Internal Forecast QC Automation\Forecast QC Automation')
from data_prep_functions import *
from mgra_input_file_automation import *

In [2]:
years = ['2022', '2026', '2029', '2032', '2035', '2040', '2050']
prev_version = '2023_11_29'
version = '2023_12_05'

# Phase 1

## 0. Data Preparation

In [3]:
server = 'ddamwsql16'
database = 'sr15_dev'
driver = 'ODBC Driver 17 for SQL Server'

engine = sql.create_engine('mssql+pyodbc://@{}/{}?trusted_connection=yes&driver={}'.format(server, database, driver))

query = """
Select *
from [sr15_staging].[2023_12_05].[mgrabase]
"""

full_mgra_data = pd.read_sql(query, con=engine.connect())
full_mgra_data

,scenario,increment,mgra,taz,luz,zip,pseudomsa,district27,pop,hhp,...,redev_ag_schools,redev_ag_roads,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable
0,0,2032,4671,2157,53,92126,3,4,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3.764135,0.000000,0.0
1,0,2032,591,1499,94,92118,2,10,79,79,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,5.104619,0.000000,0.0
2,0,2032,1778,4335,245,92021,5,13,292,292,...,0.0,0.0,0.0,0.0,0.0,0.0,3.955174,19.359395,0.808164,0.0
3,0,2032,2463,1594,2,92103,2,27,55,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,6.319781,0.000000,0.0
4,0,2032,2514,3849,75,92119,3,6,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,31.132572,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170242,0,2050,22399,4884,229,91906,8,14,13,13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,612.689264,0.000000,0.0
170243,0,2050,22400,4892,229,91906,8,14,13,13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,379.892975,0.000000,0.0
170244,0,2050,22435,4848,220,91916,8,14,112,112,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,227.739678,0.000000,0.0
170245,0,2050,22436,4848,220,91916,8,14,31,31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,92.737743,0.000000,0.0


In [4]:
for year in full_mgra_data['increment'].unique():
    full_mgra_data[full_mgra_data['increment']==year].to_csv('C:/Users/jchu/OneDrive - San Diego Association of Governments/Projects/2023/Series 15 Subregional Forecast/2023-083 MGRA Input File- Series 15 Subregional Baseline Forecast/Data/{}/{}_{}/mgra_based_input_{}_{}.csv'.format(str(year), str(year), version, str(year), version), index=False)

In [5]:
geo_dfs = {}
geo_dfs['mgra'] = mgra_output('SR15', False)
geo_dfs['mgra']['vacancy'] = geo_dfs['mgra']['hs'] - geo_dfs['mgra']['hh']
geo_dfs['mgra']['vacancy_rate'] = geo_dfs['mgra']['vacancy']/geo_dfs['mgra']['hs']
geo_dfs['mgra'] = geo_dfs['mgra'].set_index(['mgra', 'year'])
geo_dfs['mgra'].insert(0, 'geotype', 'mgra')

In [6]:
# compare columns. check if all columns are accounted for

for col in pd.read_csv(r'C:/Users/jchu/OneDrive - San Diego Association of Governments/Projects/2023/Series 15 Subregional Forecast/2023-083 MGRA Input File- Series 15 Subregional Baseline Forecast/Data/2022/2022_2023_11_10/mgra_based_input_2022_2023_11_10.csv').columns:
    if col not in list(geo_dfs['mgra'].columns):
        print(col)

year
mgra


### Geography roll-up with all forecast years (Region, Jurisdiction, CPA, LUZ)  

In [7]:
geo_levels = ['region', 'jurisdiction', 'cpa', 'luz', 'taz']
for geo in tqdm(geo_levels):
    geo_dfs[geo] = create_mgra_denorm_table(geo_dfs['mgra'].reset_index(), 
                         geo_level=geo,
                         output_folder_path=False, 
                         version='SR15',
                         series=15)
    if geo == 'cpa':
        geo_dfs[geo] = add_cpa_names(geo_dfs[geo], 15).set_index(['cpa', 'year'])
    geo_dfs[geo].insert(0, 'geotype', geo)
    display(geo_dfs[geo])
geo_dfs['jurisdiction']

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

geotype  scenario  increment       taz      luz         zip  \
region    year                                                               
San Diego 2022  region         0   49177062  61885698  2834354  2238859007   
          2026  region         0   49274346  61885698  2834354  2238859007   
          2029  region         0   49347309  61885698  2834354  2238859007   
          2032  region         0   49420272  61885698  2834354  2238859007   
          2035  region         0   49493235  61885698  2834354  2238859007   
          2040  region         0   49614840  61885698  2834354  2238859007   
          2050  region         0   49858050  61885698  2834354  2238859007   

                pseudomsa  district27      pop      hhp  ...  redev_emp_res  \
region    year                                           ...                  
San Diego 2022     111419      232154  3287306  3170895  ...    5777.011316   
          2026     111419      232154  3302237  3188816  ...    3892.168801   
          2029     111419      232154  3334675  3220394  ...    3683.472633   
          2032     111419      232154  3373033  3257421  ...    3587.083845   
          2035     111419      232154  3404362  3287304  ...    3494.090034   
          2040     111419      232154  3432211  3315776  ...    3143.145179   
          2050     111419      232154  3400250  3285676  ...    3013.373941   

                redev_emp_emp    infill_sf   infill_mf   infill_emp  \
region    year                                                        
San Diego 2022    9795.482252  5077.680910  868.121367  8754.099688   
          2026    9345.646181  2880.242699  787.866338  8577.769974   
          2029    9142.241142  2704.199250  740.414556  8377.705003   
          2032    8943.604243  2573.356699  581.599861  8171.671660   
          2035    8147.655748  2354.955800  511.746377  7703.180016   
          2040    7420.254174  2059.836740  464.330940  7140.053519   
          2050    6403.025883  1597.825257  431.514269  6582.422681   

                         dev           vac       unusable  vacancy  \
region    year                                                       
San Diego 2022  2.087883e+06  63028.828763  114774.490297    75170   
          2026  2.106848e+06  44288.003429  114774.490297    85846   
          2029  2.109682e+06  41454.160088  114774.490297    92735   
          2032  2.112020e+06  39114.916522  114774.490297    94882   
          2035  2.114745e+06  36391.996523  114774.490297    96868   
          2040  2.117018e+06  34124.406122  114774.490297    99898   
          2050  2.120085e+06  31058.964985  114774.490297   103918   

                vacancy_rate  
region    year                
San Diego 2022      0.060835  
          2026      0.066673  
          2029      0.070253  
          2032      0.070441  
          2035      0.070558  
          2040      0.070819  
          2050      0.072242  

[7 rows x 127 columns]

 20%|████████████████▊                                                                   | 1/5 [00:02<00:08,  2.18s/it]

geotype  scenario  increment     taz     luz  \
jurisdiction year                                                      
Carlsbad     2022  jurisdiction         0    1532676  294881  126916   
             2026  jurisdiction         0    1535708  294881  126916   
             2029  jurisdiction         0    1537982  294881  126916   
             2032  jurisdiction         0    1540256  294881  126916   
             2035  jurisdiction         0    1542530  294881  126916   
...                         ...       ...        ...     ...     ...   
Vista        2029  jurisdiction         0    1446677  627759  135889   
             2032  jurisdiction         0    1448816  627759  135889   
             2035  jurisdiction         0    1450955  627759  135889   
             2040  jurisdiction         0    1454520  627759  135889   
             2050  jurisdiction         0    1461650  627759  135889   

                        zip  pseudomsa  district27     pop     hhp  ...  \
jurisdiction year                                                   ...   
Carlsbad     2022  69743180       4550         859  115585  114619  ...   
             2026  69743180       4550         859  115841  114900  ...   
             2029  69743180       4550         859  119035  118086  ...   
             2032  69743180       4550         859  119702  118749  ...   
             2035  69743180       4550         859  119937  118963  ...   
...                     ...        ...         ...     ...     ...  ...   
Vista        2029  65655038       4928         713  100832   98814  ...   
             2032  65655038       4928         713  101224   99195  ...   
             2035  65655038       4928         713  101584   99499  ...   
             2040  65655038       4928         713   99947   97883  ...   
             2050  65655038       4928         713   97676   95786  ...   

                   redev_emp_res  redev_emp_emp  infill_sf  infill_mf  \
jurisdiction year                                                       
Carlsbad     2022      48.981013     266.827288   2.975080  24.947334   
             2026      28.161705     240.786677   1.212635  16.958822   
             2029      17.336878     227.419313   0.000000  16.767676   
             2032      10.089850     219.585099   0.000000   4.870878   
             2035       2.124913     210.520416   0.000000   4.870878   
...                          ...            ...        ...        ...   
Vista        2029      92.815252     107.438778  10.431880   0.000000   
             2032      91.869684     106.753358   4.910551   0.000000   
             2035      91.869684     104.393730   0.000000   0.000000   
             2040      87.702155      82.583274   0.000000   0.000000   
             2050      87.702155      62.673640   0.000000   0.000000   

                   infill_emp           dev         vac     unusable  vacancy  \
jurisdiction year                                                               
Carlsbad     2022  627.031082  18724.597352  845.282086  5471.912863     3113   
             2026  591.159335  18967.314063  613.085271  5471.912863     3499   
             2029  556.995782  19067.001469  513.397865  5471.912863     3961   
             2032  510.803826  19145.427920  434.971413  5471.912863     4045   
             2035  482.153882  19189.035361  391.363972  5471.912863     4120   
...                       ...           ...         ...          ...      ...   
Vista        2029  486.567135  11027.050357  346.154416   539.573510     1468   
             2032  483.229710  11040.105373  333.099399   539.573510     1492   
             2035  464.743406  11052.284102  320.920671   539.573510     1514   
             2040  412.180207  11104.367478  268.837294   539.573510     1604   
             2050  361.497434  11129.475974  243.728798   539.573510     1660   

                   vacancy_rate  
jurisdiction year                
Carlsbad     2022      0.064714  
     

 40%|█████████████████████████████████▌                                                  | 2/5 [00:03<00:05,  1.90s/it]

...

geotype  scenario  increment     taz       zip  pseudomsa  \
luz year                                                             
1   2022     luz         0     299256  221777  13631580        296   
    2026     luz         0     299848  221777  13631580        296   
    2029     luz         0     300292  221777  13631580        296   
    2032     luz         0     300736  221777  13631580        296   
    2035     luz         0     301180  221777  13631580        296   
...          ...       ...        ...     ...       ...        ...   
245 2029     luz         0      79131  169137   3588811        195   
    2032     luz         0      79248  169137   3588811        195   
    2035     luz         0      79365  169137   3588811        195   
    2040     luz         0      79560  169137   3588811        195   
    2050     luz         0      79950  169137   3588811        195   

          district27    pop    hhp  gq  ...  redev_emp_res  redev_emp_emp  \
luz year                                ...                                 
1   2022        3996   7233   7191  42  ...       0.115001      17.192360   
    2026        3996   7515   7474  41  ...       0.000000      17.192360   
    2029        3996   7596   7555  41  ...       0.000000      17.192360   
    2032        3996   7621   7580  41  ...       0.000000      17.192360   
    2035        3996   7735   7694  41  ...       0.000000      17.192360   
...              ...    ...    ...  ..  ...            ...            ...   
245 2029         507  10707  10637  70  ...       4.691822      93.587226   
    2032         507  10646  10576  70  ...       4.691822      92.369799   
    2035         507  10668  10597  71  ...       4.491585      87.975826   
    2040         507  11134  11064  70  ...       3.721173      73.319766   
    2050         507  10911  10849  62  ...       3.470877      47.618641   

          infill_sf  infill_mf  infill_emp         dev        vac   unusable  \
luz year                                                                       
1   2022   0.000000   0.000000    6.912115  877.007383  18.844594  59.487941   
    2026   0.000000   0.000000    6.912115  883.270231  13.729838  59.487941   
    2029   0.000000   0.000000    6.912115  883.423150  13.576919  59.487941   
    2032   0.000000   0.000000    6.912115  883.609566  13.390503  59.487941   
    2035   0.000000   0.000000    6.912115  883.867322  13.132746  59.487941   
...             ...        ...         ...         ...        ...        ...   
245 2029  77.829521  27.032324    8.162603  687.867490  19.036380   0.000000   
    2032  74.730239  26.902801    8.162603  688.179774  18.724097   0.000000   
    2035  68.224587  25.996143    8.162603  689.502872  17.400998   0.000000   
    2040   0.000000  24.441871    8.162603  696.191335  10.712536   0.000000   
    2050   0.000000  22.499032    8.162603  698.968235   7.935635   0.000000   

          vacancy  vacancy_rate  
luz year                         
1   2022      188      0.049292  
    2026      217      0.053646  
    2029      236      0.057129  
    2032      239      0.057259  
    2035      242      0.056808  
...           ...           ...  
245 2029       56      0.014452  
    2032       56      0.014396  
    2035       55      0.013988  
    2040       67      0.015983  
    2050       70      0.016556  

[1715 rows x 126 columns]

 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:09<00:02,  2.36s/it]

geotype  scenario  increment   luz     zip  pseudomsa  district27  \
taz  year                                                                     
13   2022     taz         0       8088   676  370071         24           4   
     2026     taz         0       8104   676  370071         24           4   
     2029     taz         0       8116   676  370071         24           4   
     2032     taz         0       8128   676  370071         24           4   
     2035     taz         0       8140   676  370071         24           4   
...           ...       ...        ...   ...     ...        ...         ...   
4947 2029     taz         0      16232  1808  735472         64         112   
     2032     taz         0      16256  1808  735472         64         112   
     2035     taz         0      16280  1808  735472         64         112   
     2040     taz         0      16320  1808  735472         64         112   
     2050     taz         0      16400  1808  735472         64         112   

            pop   hhp    gq  ...  redev_emp_res  redev_emp_emp  infill_sf  \
taz  year                    ...                                            
13   2022  4646  2859  1787  ...            0.0            0.0        0.0   
     2026  4591  2831  1760  ...            0.0            0.0        0.0   
     2029  4565  2796  1769  ...            0.0            0.0        0.0   
     2032  4550  2767  1783  ...            0.0            0.0        0.0   
     2035  4536  2740  1796  ...            0.0            0.0        0.0   
...         ...   ...   ...  ...            ...            ...        ...   
4947 2029     7     7     0  ...            0.0            0.0        0.0   
     2032     7     7     0  ...            0.0            0.0        0.0   
     2035     7     7     0  ...            0.0            0.0        0.0   
     2040     7     7     0  ...            0.0            0.0        0.0   
     2050     7     7     0  ...            0.0            0.0        0.0   

           infill_mf  infill_emp          dev        vac  unusable  vacancy  \
taz  year                                                                     
13   2022        0.0         0.0  4889.004243    0.00000       0.0       83   
     2026        0.0         0.0  4889.004243    0.00000       0.0       66   
     2029        0.0         0.0  4889.004243    0.00000       0.0       67   
     2032        0.0         0.0  4889.004243    0.00000       0.0       67   
     2035        0.0         0.0  4889.004243    0.00000       0.0       67   
...              ...         ...          ...        ...       ...      ...   
4947 2029        0.0         0.0   960.073851  270.55383       0.0        5   
     2032        0.0         0.0   960.073851  270.55383       0.0        5   
     2035        0.0         0.0   960.073851  270.55383       0.0        5   
     2040        0.0         0.0   960.073851  270.55383       0.0        5   
     2050        0.0         0.0   960.073851  270.55383       0.0        5   

           vacancy_rate  
taz  year                
13   2022      0.083585  
     2026      0.066465  
     2029      0.067472  
     2032      0.067472  
     2035      0.067472  
...                 ...  
4947 2029      0.625000  
     2032      0.625000  
     2035      0.625000  
     2040      0.625000  
     2050      0.625000  

[34545 rows x 126 columns]

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:11<00:00,  2.27s/it]


geotype  scenario  increment     taz     luz  \
jurisdiction year                                                      
Carlsbad     2022  jurisdiction         0    1532676  294881  126916   
             2026  jurisdiction         0    1535708  294881  126916   
             2029  jurisdiction         0    1537982  294881  126916   
             2032  jurisdiction         0    1540256  294881  126916   
             2035  jurisdiction         0    1542530  294881  126916   
...                         ...       ...        ...     ...     ...   
Vista        2029  jurisdiction         0    1446677  627759  135889   
             2032  jurisdiction         0    1448816  627759  135889   
             2035  jurisdiction         0    1450955  627759  135889   
             2040  jurisdiction         0    1454520  627759  135889   
             2050  jurisdiction         0    1461650  627759  135889   

                        zip  pseudomsa  district27     pop     hhp  ...  \
jurisdiction year                                                   ...   
Carlsbad     2022  69743180       4550         859  115585  114619  ...   
             2026  69743180       4550         859  115841  114900  ...   
             2029  69743180       4550         859  119035  118086  ...   
             2032  69743180       4550         859  119702  118749  ...   
             2035  69743180       4550         859  119937  118963  ...   
...                     ...        ...         ...     ...     ...  ...   
Vista        2029  65655038       4928         713  100832   98814  ...   
             2032  65655038       4928         713  101224   99195  ...   
             2035  65655038       4928         713  101584   99499  ...   
             2040  65655038       4928         713   99947   97883  ...   
             2050  65655038       4928         713   97676   95786  ...   

                   redev_emp_res  redev_emp_emp  infill_sf  infill_mf  \
jurisdiction year                                                       
Carlsbad     2022      48.981013     266.827288   2.975080  24.947334   
             2026      28.161705     240.786677   1.212635  16.958822   
             2029      17.336878     227.419313   0.000000  16.767676   
             2032      10.089850     219.585099   0.000000   4.870878   
             2035       2.124913     210.520416   0.000000   4.870878   
...                          ...            ...        ...        ...   
Vista        2029      92.815252     107.438778  10.431880   0.000000   
             2032      91.869684     106.753358   4.910551   0.000000   
             2035      91.869684     104.393730   0.000000   0.000000   
             2040      87.702155      82.583274   0.000000   0.000000   
             2050      87.702155      62.673640   0.000000   0.000000   

                   infill_emp           dev         vac     unusable  vacancy  \
jurisdiction year                                                               
Carlsbad     2022  627.031082  18724.597352  845.282086  5471.912863     3113   
             2026  591.159335  18967.314063  613.085271  5471.912863     3499   
             2029  556.995782  19067.001469  513.397865  5471.912863     3961   
             2032  510.803826  19145.427920  434.971413  5471.912863     4045   
             2035  482.153882  19189.035361  391.363972  5471.912863     4120   
...                       ...           ...         ...          ...      ...   
Vista        2029  486.567135  11027.050357  346.154416   539.573510     1468   
             2032  483.229710  11040.105373  333.099399   539.573510     1492   
             2035  464.743406  11052.284102  320.920671   539.573510     1514   
             2040  412.180207  11104.367478  268.837294   539.573510     1604   
             2050  361.497434  11129.475974  243.728798   539.573510     1660   

                   vacancy_rate  
jurisdiction year                
Carlsbad     2022      0.064714  
     

In [8]:
# consolidated files:
writer = pd.ExcelWriter(r'Data/mgra15_based_input_consolidated_{}.xlsx'.format(version), engine='xlsxwriter')
for geo, df in tqdm(geo_dfs.items()):
    display(df)
    df.reset_index().sort_values([geo, 'year']).to_excel(writer, sheet_name = '{}'.format(geo), index=False)
writer.save()

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

,,geotype,scenario,increment,taz,luz,zip,pseudomsa,district27,pop,hhp,...,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate
mgra,year,,,,,,,,,,,,,,,,,,,,,
1336,2022,mgra,0,2022,2700,96,91950,2,10,64,64,...,0.0,0.0,0.0,0.0,0.000000,3.587805,0.000000,0.0,0,0.000000
1329,2022,mgra,0,2022,720,34,92107,2,26,13,13,...,0.0,0.0,0.0,0.0,0.000000,1.620034,0.000000,0.0,0,0.000000
4925,2022,mgra,0,2022,1914,65,92123,3,5,0,0,...,0.0,0.0,0.0,0.0,0.000000,44.651054,1.931806,0.0,0,NaN
1331,2022,mgra,0,2022,371,151,92007,6,2,151,151,...,0.0,0.0,0.0,0.0,0.070214,7.951683,0.000000,0.0,9,0.111111
4918,2022,mgra,0,2022,3294,101,91910,4,11,241,241,...,0.0,0.0,0.0,0.0,0.000000,20.618079,0.000000,0.0,3,0.040000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22399,2050,mgra,0,2050,4884,229,91906,8,14,13,13,...,0.0,0.0,0.0,0.0,0.000000,612.689264,0.000000,0.0,4,0.444444
22400,2050,mgra,0,2050,4892,229,91906,8,14,13,13,...,0.0,0.0,0.0,0.0,0.000000,379.892975,0.000000,0.0,5,0.500000
22435,2050,mgra,0,2050,4848,220,91916,8,14,112,112,...,0.0,0.0,0.0,0.0,0.000000,227.739678,0.000000,0.0,1,0.021277


 17%|█████████████▊                                                                     | 1/6 [02:27<12:15, 147.09s/it]

geotype  scenario  increment       taz      luz         zip  \
region    year                                                               
San Diego 2022  region         0   49177062  61885698  2834354  2238859007   
          2026  region         0   49274346  61885698  2834354  2238859007   
          2029  region         0   49347309  61885698  2834354  2238859007   
          2032  region         0   49420272  61885698  2834354  2238859007   
          2035  region         0   49493235  61885698  2834354  2238859007   
          2040  region         0   49614840  61885698  2834354  2238859007   
          2050  region         0   49858050  61885698  2834354  2238859007   

                pseudomsa  district27      pop      hhp  ...  redev_emp_res  \
region    year                                           ...                  
San Diego 2022     111419      232154  3287306  3170895  ...    5777.011316   
          2026     111419      232154  3302237  3188816  ...    3892.168801   
          2029     111419      232154  3334675  3220394  ...    3683.472633   
          2032     111419      232154  3373033  3257421  ...    3587.083845   
          2035     111419      232154  3404362  3287304  ...    3494.090034   
          2040     111419      232154  3432211  3315776  ...    3143.145179   
          2050     111419      232154  3400250  3285676  ...    3013.373941   

                redev_emp_emp    infill_sf   infill_mf   infill_emp  \
region    year                                                        
San Diego 2022    9795.482252  5077.680910  868.121367  8754.099688   
          2026    9345.646181  2880.242699  787.866338  8577.769974   
          2029    9142.241142  2704.199250  740.414556  8377.705003   
          2032    8943.604243  2573.356699  581.599861  8171.671660   
          2035    8147.655748  2354.955800  511.746377  7703.180016   
          2040    7420.254174  2059.836740  464.330940  7140.053519   
          2050    6403.025883  1597.825257  431.514269  6582.422681   

                         dev           vac       unusable  vacancy  \
region    year                                                       
San Diego 2022  2.087883e+06  63028.828763  114774.490297    75170   
          2026  2.106848e+06  44288.003429  114774.490297    85846   
          2029  2.109682e+06  41454.160088  114774.490297    92735   
          2032  2.112020e+06  39114.916522  114774.490297    94882   
          2035  2.114745e+06  36391.996523  114774.490297    96868   
          2040  2.117018e+06  34124.406122  114774.490297    99898   
          2050  2.120085e+06  31058.964985  114774.490297   103918   

                vacancy_rate  
region    year                
San Diego 2022      0.060835  
          2026      0.066673  
          2029      0.070253  
          2032      0.070441  
          2035      0.070558  
          2040      0.070819  
          2050      0.072242  

[7 rows x 127 columns]

geotype  scenario  increment     taz     luz  \
jurisdiction year                                                      
Carlsbad     2022  jurisdiction         0    1532676  294881  126916   
             2026  jurisdiction         0    1535708  294881  126916   
             2029  jurisdiction         0    1537982  294881  126916   
             2032  jurisdiction         0    1540256  294881  126916   
             2035  jurisdiction         0    1542530  294881  126916   
...                         ...       ...        ...     ...     ...   
Vista        2029  jurisdiction         0    1446677  627759  135889   
             2032  jurisdiction         0    1448816  627759  135889   
             2035  jurisdiction         0    1450955  627759  135889   
             2040  jurisdiction         0    1454520  627759  135889   
             2050  jurisdiction         0    1461650  627759  135889   

                        zip  pseudomsa  district27     pop     hhp  ...  \
jurisdiction year                                                   ...   
Carlsbad     2022  69743180       4550         859  115585  114619  ...   
             2026  69743180       4550         859  115841  114900  ...   
             2029  69743180       4550         859  119035  118086  ...   
             2032  69743180       4550         859  119702  118749  ...   
             2035  69743180       4550         859  119937  118963  ...   
...                     ...        ...         ...     ...     ...  ...   
Vista        2029  65655038       4928         713  100832   98814  ...   
             2032  65655038       4928         713  101224   99195  ...   
             2035  65655038       4928         713  101584   99499  ...   
             2040  65655038       4928         713   99947   97883  ...   
             2050  65655038       4928         713   97676   95786  ...   

                   redev_emp_res  redev_emp_emp  infill_sf  infill_mf  \
jurisdiction year                                                       
Carlsbad     2022      48.981013     266.827288   2.975080  24.947334   
             2026      28.161705     240.786677   1.212635  16.958822   
             2029      17.336878     227.419313   0.000000  16.767676   
             2032      10.089850     219.585099   0.000000   4.870878   
             2035       2.124913     210.520416   0.000000   4.870878   
...                          ...            ...        ...        ...   
Vista        2029      92.815252     107.438778  10.431880   0.000000   
             2032      91.869684     106.753358   4.910551   0.000000   
             2035      91.869684     104.393730   0.000000   0.000000   
             2040      87.702155      82.583274   0.000000   0.000000   
             2050      87.702155      62.673640   0.000000   0.000000   

                   infill_emp           dev         vac     unusable  vacancy  \
jurisdiction year                                                               
Carlsbad     2022  627.031082  18724.597352  845.282086  5471.912863     3113   
             2026  591.159335  18967.314063  613.085271  5471.912863     3499   
             2029  556.995782  19067.001469  513.397865  5471.912863     3961   
             2032  510.803826  19145.427920  434.971413  5471.912863     4045   
             2035  482.153882  19189.035361  391.363972  5471.912863     4120   
...                       ...           ...         ...          ...      ...   
Vista        2029  486.567135  11027.050357  346.154416   539.573510     1468   
             2032  483.229710  11040.105373  333.099399   539.573510     1492   
             2035  464.743406  11052.284102  320.920671   539.573510     1514   
             2040  412.180207  11104.367478  268.837294   539.573510     1604   
             2050  361.497434  11129.475974  243.728798   539.573510     1660   

                   vacancy_rate  
jurisdiction year                
Carlsbad     2022      0.064714  
     

...

geotype  scenario  increment       taz      luz        zip  \
cpa  year                                                              
0    2022     cpa         0   18270792  19576400  1343072  831452093   
     2026     cpa         0   18306936  19576400  1343072  831452093   
     2029     cpa         0   18334044  19576400  1343072  831452093   
     2032     cpa         0   18361152  19576400  1343072  831452093   
     2035     cpa         0   18388260  19576400  1343072  831452093   
...           ...       ...        ...       ...      ...        ...   
1999 2029     cpa         0      56812     87662     2730    2574733   
     2032     cpa         0      56896     87662     2730    2574733   
     2035     cpa         0      56980     87662     2730    2574733   
     2040     cpa         0      57120     87662     2730    2574733   
     2050     cpa         0      57400     87662     2730    2574733   

           pseudomsa  district27      pop      hhp  ...  redev_emp_emp  \
cpa  year                                           ...                  
0    2022      47673       49808  1399346  1372115  ...    1811.178538   
     2026      47673       49808  1399803  1373636  ...    1656.061357   
     2029      47673       49808  1417019  1390503  ...    1597.243749   
     2032      47673       49808  1451434  1424008  ...    1517.895996   
     2035      47673       49808  1466553  1438544  ...    1463.489250   
...              ...         ...      ...      ...  ...            ...   
1999 2029         60         274     2539     2539  ...      64.510573   
     2032         60         274     2537     2537  ...      64.306363   
     2035         60         274     2577     2577  ...      63.871183   
     2040         60         274     2525     2525  ...      62.786514   
     2050         60         274     2584     2584  ...      59.811422   

           infill_sf   infill_mf   infill_emp            dev           vac  \
cpa  year                                                                    
0    2022  70.521071  559.760262  4448.567459  177412.265136  14760.720904   
     2026  29.169509  506.366823  4320.317121  180853.203741  11394.838346   
     2029  22.780979  461.758200  4171.256838  182341.525777   9906.516309   
     2032   7.672304  407.280975  4012.085057  183685.580225   8561.040730   
     2035   2.510639  342.874423  3688.324460  184656.797300   7590.854737   
...              ...         ...          ...            ...           ...   
1999 2029   2.146079    0.000000     0.000000     366.552780      8.717923   
     2032   2.146079    0.000000     0.000000     366.927985      8.342718   
     2035   2.056487    0.000000     0.000000     367.817233      7.453470   
     2040   2.056487    0.000000     0.000000     368.002352      7.268351   
     2050   0.000000    0.000000     0.000000     371.018476      4.252227   

               unusable  vacancy  vacancy_rate        cpa_name  
cpa  year                                                       
0    2022  31874.072908    26057      0.051809      Not in CPA  
     2026  31874.072908    29158      0.056224      Not in CPA  
     2029  31874.072908    31607      0.059406      Not in CPA  
     2032  31874.072908    32627      0.059422      Not in CPA  
     2035  31874.072908    33390      0.059654      Not in CPA  
...                 ...      ...           ...             ...  
1999 2029      0.000000       40      0.053836  County Islands  
     2032      0.000000       40      0.053262  County Islands  
     2035      0.000000       40      0.051881  County Islands  
     2040      0.000000       42      0.054124  County Islands  
     2050      0.000000       46      0.055489  County Islands  

[616 rows x 128 columns]

geotype  scenario  increment     taz       zip  pseudomsa  \
luz year                                                             
1   2022     luz         0     299256  221777  13631580        296   
    2026     luz         0     299848  221777  13631580        296   
    2029     luz         0     300292  221777  13631580        296   
    2032     luz         0     300736  221777  13631580        296   
    2035     luz         0     301180  221777  13631580        296   
...          ...       ...        ...     ...       ...        ...   
245 2029     luz         0      79131  169137   3588811        195   
    2032     luz         0      79248  169137   3588811        195   
    2035     luz         0      79365  169137   3588811        195   
    2040     luz         0      79560  169137   3588811        195   
    2050     luz         0      79950  169137   3588811        195   

          district27    pop    hhp  gq  ...  redev_emp_res  redev_emp_emp  \
luz year                                ...                                 
1   2022        3996   7233   7191  42  ...       0.115001      17.192360   
    2026        3996   7515   7474  41  ...       0.000000      17.192360   
    2029        3996   7596   7555  41  ...       0.000000      17.192360   
    2032        3996   7621   7580  41  ...       0.000000      17.192360   
    2035        3996   7735   7694  41  ...       0.000000      17.192360   
...              ...    ...    ...  ..  ...            ...            ...   
245 2029         507  10707  10637  70  ...       4.691822      93.587226   
    2032         507  10646  10576  70  ...       4.691822      92.369799   
    2035         507  10668  10597  71  ...       4.491585      87.975826   
    2040         507  11134  11064  70  ...       3.721173      73.319766   
    2050         507  10911  10849  62  ...       3.470877      47.618641   

          infill_sf  infill_mf  infill_emp         dev        vac   unusable  \
luz year                                                                       
1   2022   0.000000   0.000000    6.912115  877.007383  18.844594  59.487941   
    2026   0.000000   0.000000    6.912115  883.270231  13.729838  59.487941   
    2029   0.000000   0.000000    6.912115  883.423150  13.576919  59.487941   
    2032   0.000000   0.000000    6.912115  883.609566  13.390503  59.487941   
    2035   0.000000   0.000000    6.912115  883.867322  13.132746  59.487941   
...             ...        ...         ...         ...        ...        ...   
245 2029  77.829521  27.032324    8.162603  687.867490  19.036380   0.000000   
    2032  74.730239  26.902801    8.162603  688.179774  18.724097   0.000000   
    2035  68.224587  25.996143    8.162603  689.502872  17.400998   0.000000   
    2040   0.000000  24.441871    8.162603  696.191335  10.712536   0.000000   
    2050   0.000000  22.499032    8.162603  698.968235   7.935635   0.000000   

          vacancy  vacancy_rate  
luz year                         
1   2022      188      0.049292  
    2026      217      0.053646  
    2029      236      0.057129  
    2032      239      0.057259  
    2035      242      0.056808  
...           ...           ...  
245 2029       56      0.014452  
    2032       56      0.014396  
    2035       55      0.013988  
    2040       67      0.015983  
    2050       70      0.016556  

[1715 rows x 126 columns]

 83%|██████████████████████████████████████████████████████████████████████              | 5/6 [02:28<00:18, 18.98s/it]

geotype  scenario  increment   luz     zip  pseudomsa  district27  \
taz  year                                                                     
13   2022     taz         0       8088   676  370071         24           4   
     2026     taz         0       8104   676  370071         24           4   
     2029     taz         0       8116   676  370071         24           4   
     2032     taz         0       8128   676  370071         24           4   
     2035     taz         0       8140   676  370071         24           4   
...           ...       ...        ...   ...     ...        ...         ...   
4947 2029     taz         0      16232  1808  735472         64         112   
     2032     taz         0      16256  1808  735472         64         112   
     2035     taz         0      16280  1808  735472         64         112   
     2040     taz         0      16320  1808  735472         64         112   
     2050     taz         0      16400  1808  735472         64         112   

            pop   hhp    gq  ...  redev_emp_res  redev_emp_emp  infill_sf  \
taz  year                    ...                                            
13   2022  4646  2859  1787  ...            0.0            0.0        0.0   
     2026  4591  2831  1760  ...            0.0            0.0        0.0   
     2029  4565  2796  1769  ...            0.0            0.0        0.0   
     2032  4550  2767  1783  ...            0.0            0.0        0.0   
     2035  4536  2740  1796  ...            0.0            0.0        0.0   
...         ...   ...   ...  ...            ...            ...        ...   
4947 2029     7     7     0  ...            0.0            0.0        0.0   
     2032     7     7     0  ...            0.0            0.0        0.0   
     2035     7     7     0  ...            0.0            0.0        0.0   
     2040     7     7     0  ...            0.0            0.0        0.0   
     2050     7     7     0  ...            0.0            0.0        0.0   

           infill_mf  infill_emp          dev        vac  unusable  vacancy  \
taz  year                                                                     
13   2022        0.0         0.0  4889.004243    0.00000       0.0       83   
     2026        0.0         0.0  4889.004243    0.00000       0.0       66   
     2029        0.0         0.0  4889.004243    0.00000       0.0       67   
     2032        0.0         0.0  4889.004243    0.00000       0.0       67   
     2035        0.0         0.0  4889.004243    0.00000       0.0       67   
...              ...         ...          ...        ...       ...      ...   
4947 2029        0.0         0.0   960.073851  270.55383       0.0        5   
     2032        0.0         0.0   960.073851  270.55383       0.0        5   
     2035        0.0         0.0   960.073851  270.55383       0.0        5   
     2040        0.0         0.0   960.073851  270.55383       0.0        5   
     2050        0.0         0.0   960.073851  270.55383       0.0        5   

           vacancy_rate  
taz  year                
13   2022      0.083585  
     2026      0.066465  
     2029      0.067472  
     2032      0.067472  
     2035      0.067472  
...                 ...  
4947 2029      0.625000  
     2032      0.625000  
     2035      0.625000  
     2040      0.625000  
     2050      0.625000  

[34545 rows x 126 columns]

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [02:58<00:00, 29.68s/it]


In [9]:
# write year data into excel
for year in tqdm(years):
    writer = pd.ExcelWriter(r'Data/{}/mgra15_based_input_{}_{}.xlsx'.format(year, year, version), engine='xlsxwriter')
    for geo, df in tqdm(geo_dfs.items()):
        output = df.reset_index()
        output = output[output['year'].isin([year])].reset_index(drop=True)
        if 'taz' in list(output.columns)and geo != 'taz':
            output = output.drop('taz', axis=1)
        if 'LUZ' in list(output.columns):
            output = output.drop('LUZ', axis=1)
        output.insert(1, 'geozone', output[geo])
        display(output)
        output.to_excel(writer, sheet_name = '{}'.format(geo), index=False)
    writer.save()

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

,mgra,geozone,year,geotype,scenario,increment,luz,zip,pseudomsa,district27,...,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate
0,1336,1336,2022,mgra,0,2022,96,91950,2,10,...,0.0,0.0,0.0,0.0,0.000000,3.587805,0.000000,0.0,0,0.000000
1,1329,1329,2022,mgra,0,2022,34,92107,2,26,...,0.0,0.0,0.0,0.0,0.000000,1.620034,0.000000,0.0,0,0.000000
2,4925,4925,2022,mgra,0,2022,65,92123,3,5,...,0.0,0.0,0.0,0.0,0.000000,44.651054,1.931806,0.0,0,NaN
3,1331,1331,2022,mgra,0,2022,151,92007,6,2,...,0.0,0.0,0.0,0.0,0.070214,7.951683,0.000000,0.0,9,0.111111
4,4918,4918,2022,mgra,0,2022,101,91910,4,11,...,0.0,0.0,0.0,0.0,0.000000,20.618079,0.000000,0.0,3,0.040000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24316,14411,14411,2022,mgra,0,2022,107,92019,5,13,...,0.0,0.0,0.0,0.0,0.000000,45.631507,0.000000,0.0,1,0.066667
24317,14497,14497,2022,mgra,0,2022,239,92019,5,13,...,0.0,0.0,0.0,0.0,1.739239,7.289608,0.000000,0.0,0,0.000000
24318,493,493,2022,mgra,0,2022,2,92103,2,27,...,0.0,0.0,0.0,0.0,0.000000,3.145849,0.114938,0.0,7,0.101449
24319,14461,14461,2022,mgra,0,2022,108,92019,5,13,...,0.0,0.0,0.0,0.0,0.000000,29.604987,0.000000,0.0,10,0.061728



 17%|██████████████                                                                      | 1/6 [00:20<01:40, 20.18s/it]

,region,geozone,year,geotype,scenario,increment,luz,zip,pseudomsa,district27,...,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate
0,San Diego,San Diego,2022,region,0,49177062,2834354,2238859007,111419,232154,...,5777.011316,9795.482252,5077.68091,868.121367,8754.099688,2.087883e+06,63028.828763,114774.490297,75170,0.060835


,jurisdiction,geozone,year,geotype,scenario,increment,luz,zip,pseudomsa,district27,...,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate
0,Carlsbad,Carlsbad,2022,jurisdiction,0,1532676,126916,69743180,4550,859,...,48.981013,266.827288,2.975080,24.947334,627.031082,1.872460e+04,845.282086,5471.912863,3113,0.064714
1,Chula Vista,Chula Vista,2022,jurisdiction,0,2173650,112956,98805269,4299,12401,...,245.136647,319.726038,0.503478,8.292870,499.112329,2.774602e+04,2730.834133,3057.663242,2731,0.030984
2,Coronado,Coronado,2022,jurisdiction,0,450906,23634,20542192,446,2243,...,2.190672,122.140115,9.515815,0.508823,26.040527,9.757778e+03,24.952730,309.573962,2210,0.228660
3,Del Mar,Del Mar,2022,jurisdiction,0,133452,2904,6072924,198,148,...,3.042198,9.985388,0.000000,0.000000,0.378786,1.000006e+03,33.310773,102.908124,655,0.249144
4,El Cajon,El Cajon,2022,jurisdiction,0,1188936,82262,54107791,2940,7644,...,23.834850,63.628054,2.520019,4.246081,376.072664,8.846823e+03,272.873824,155.789973,1194,0.032632
5,Encinitas,Encinitas,2022,jurisdiction,0,1660062,123345,75548908,4926,1642,...,9.670661,52.607779,1.557644,0.000000,162.988627,9.839552e+03,426.598975,2246.207532,2068,0.077555
6,Escondido,Escondido,2022,jurisdiction,0,1900680,192371,86504804,6580,2820,...,64.587804,129.064203,5.776852,0.000000,560.361195,1.724417e+04,1665.539483,5034.396925,1810,0.036566
7,Imperial Beach,Imperial Beach,2022,jurisdiction,0,307344,29492,13973664,608,1672,...,3.533559,46.196002,0.130431,0.000000,10.468602,1.733914e+03,11.617292,1063.725145,714,0.071158
8,La Mesa,La Mesa,2022,jurisdiction,0,948318,51501,43120619,2345,4066,...,12.180635,6.519877,0.527566,0.000000,142.468202,5.676175e+03,122.549718,3.244729,1229,0.046792
9,Lemon Grove,Lemon Grove,2022,jurisdiction,0,363960,11559,16550100,879,1806,...,12.998008,1.001917,1.646096,0.000000,34.942079,2.384757e+03,63.252898,0.048723,337,0.036686


...

,cpa,geozone,year,geotype,scenario,increment,luz,zip,pseudomsa,district27,...,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate,cpa_name
0,0,0,2050,cpa,0,18523800,1343072,831452093,47673,49808,...,1199.593601,0.0,291.212759,2934.515819,186342.184185,5912.035953,31874.072908,36407,0.062355,Not in CPA
1,1401,1401,2050,cpa,0,59450,841,2670998,58,270,...,0.000000,0.0,0.000000,68.980657,1256.647123,30.832045,0.000000,12,0.923077,Balboa Park
2,1402,1402,2050,cpa,0,114800,1291,5158328,112,715,...,15.106046,0.0,0.184453,30.680444,550.013693,25.967361,0.013872,250,0.134048,Barrio Logan
3,1403,1403,2050,cpa,0,137350,9179,6172576,201,268,...,16.665674,0.0,0.000000,60.058419,1486.512519,0.895999,78.723801,205,0.032695,Carmel Mountain Ranch
4,1404,1404,2050,cpa,0,701100,9564,31498626,366,5163,...,101.659177,0.0,0.000000,59.381235,1468.932909,42.265611,0.043054,9945,0.172536,Downtown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,1953,1953,2050,cpa,0,1053700,88160,47302443,3585,1398,...,0.000000,0.0,4.796455,0.000000,25628.899340,6.897847,5896.458971,967,0.055549,Fallbrook
84,1954,1954,2050,cpa,0,96350,8180,4328274,288,47,...,0.000000,0.0,0.000000,0.000000,150245.997039,0.000000,34.539931,666,0.087956,Pendleton-De Luz
85,1955,1955,2050,cpa,0,219350,18725,9846996,749,1381,...,0.000000,0.0,0.000000,0.000000,5366.997447,0.000000,589.345390,74,0.097754,Rainbow
86,1998,1998,2050,cpa,0,14350,895,644305,35,98,...,0.000000,0.0,0.000000,0.000000,2620.091211,0.000000,41.454752,1,0.003472,Barona



 67%|████████████████████████████████████████████████████████                            | 4/6 [00:19<00:07,  3.70s/it]

,luz,geozone,year,geotype,scenario,increment,zip,pseudomsa,district27,pop,...,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate
0,1,1,2050,luz,0,303400,13631580,296,3996,7132,...,0.000000,17.192360,0.0,0.000000,6.912115,883.949940,13.050129,59.487941,533,0.122332
1,2,2,2050,luz,0,467400,20999350,456,6156,19620,...,0.517866,0.671545,0.0,0.089669,2.513927,879.252141,12.858133,22.277557,623,0.048905
2,3,3,2050,luz,0,284950,12802577,278,2404,22329,...,1.060529,1.987185,0.0,0.133961,4.108407,1023.811261,13.560552,68.672775,385,0.027555
3,4,4,2050,luz,0,508400,22842392,496,1984,43471,...,0.361248,0.000000,0.0,0.160697,0.000000,1487.453727,4.255203,5.674370,2356,0.082329
4,5,5,2050,luz,0,264450,11881946,386,875,43798,...,8.610671,59.474927,0.0,2.486945,170.657537,2926.896508,207.199967,84.335003,3030,0.112280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,241,241,2050,luz,0,225500,10129220,770,110,11705,...,0.000000,0.000000,0.0,0.000000,0.000000,1751.858892,42.949702,28.649474,121,0.028384
241,242,242,2050,luz,0,32800,1472592,48,267,4898,...,0.000000,34.226770,0.0,0.000000,36.880099,387.804901,27.248322,0.000000,575,0.192051
242,243,243,2050,luz,0,55350,2487267,81,432,1540,...,0.000000,60.211690,0.0,0.000000,74.380437,788.405251,37.026398,71.734428,44,0.057667
243,244,244,2050,luz,0,92250,4145477,135,720,9139,...,0.000000,9.687128,0.0,0.000000,68.406062,670.007353,10.098877,19.166255,910,0.170030


,taz,geozone,year,geotype,scenario,increment,luz,zip,pseudomsa,district27,...,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate
0,13,13,2050,taz,0,8200,676,370071,24,4,...,0.0,0.000000,0.0,0.000000,0.000000,4889.004243,0.000000,0.000000,69,0.069486
1,14,14,2050,taz,0,18450,1521,828495,54,9,...,0.0,0.000000,0.0,0.000000,0.000000,25603.832503,0.000000,0.000000,123,0.490040
2,15,15,2050,taz,0,6150,720,276165,18,3,...,0.0,0.000000,0.0,0.000000,0.000000,1108.992996,0.000000,0.000000,24,0.079470
3,16,16,2050,taz,0,6150,483,276162,18,3,...,0.0,0.000000,0.0,1.208995,1.562854,122.196625,0.023044,9.290139,25,0.373134
4,17,17,2050,taz,0,34850,2873,1564935,102,17,...,0.0,0.000000,0.0,0.000000,0.000000,73140.455309,0.000000,0.000000,254,0.158058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4930,4943,4943,2050,taz,0,12300,1350,552235,48,84,...,0.0,459.067789,0.0,0.000000,0.000000,94142.693533,0.000000,0.000000,48,0.615385
4931,4944,4944,2050,taz,0,8200,904,367736,32,56,...,0.0,4.215455,0.0,0.000000,0.000000,364.012742,43.630083,0.000000,6,0.037267
4932,4945,4945,2050,taz,0,4100,452,183868,16,28,...,0.0,0.000000,0.0,0.000000,0.000000,311.731702,694.122268,0.000000,3,0.428571
4933,4946,4946,2050,taz,0,8200,884,368016,32,56,...,0.0,0.000000,0.0,0.000000,0.000000,35300.498583,0.000000,0.000000,0,NaN



100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [04:24<00:00, 37.84s/it]


### 2026 and 2029 Diff file- Series 14 (Jurisdiction, CPA, LUZ, Region) 

In [10]:
# get series 14 data
series14_df = {}

for geo in tqdm(geo_dfs.keys()):
    if geo == 'mgra':
        series14_df['mgra'] = mgra_output('41', False)
        series14_df['mgra']['vacancy'] = series14_df['mgra']['hs'] - series14_df['mgra']['hh']
        series14_df['mgra']['vacancy_rate'] = series14_df['mgra']['vacancy']/series14_df['mgra']['hs']
    else:
        sr14 = series14_df['mgra'].copy()
        sr14 = sr14[sr14['year'].astype(int)>2025]
        series14_df[geo] = create_mgra_denorm_table(mgra_denorm_path=sr14, 
                                         geo_level=geo, 
                                         output_folder_path=False, 
                                         version='SR14',
                                         series=14)
    if geo == 'cpa':
        series14_df[geo] = add_cpa_names(series14_df[geo].reset_index(), 14).set_index(['cpa', 'year'])
        
    series14_df[geo].insert(0, 'geotype', geo)

series14_df['cpa']

 50%|██████████████████████████████████████████                                          | 3/6 [00:09<00:08,  2.85s/it]C:\Users\jchu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3364: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):
100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:19<00:00,  3.32s/it]


geotype  index        taz       hs   hs_sf    hs_mf  hs_mh  \
cpa    year                                                              
1442.0 2026     cpa  280.0  1633143.0  26556.0  7861.0  18695.0    0.0   
       2029     cpa  281.0  1633143.0  28224.0  7892.0  20332.0    0.0   
       2030     cpa  282.0  1633143.0  28794.0  7892.0  20902.0    0.0   
       2032     cpa  283.0  1633143.0  29056.0  7896.0  21160.0    0.0   
       2035     cpa  284.0  1633143.0  29523.0  7897.0  21626.0    0.0   
...             ...    ...        ...      ...     ...      ...    ...   
1906.0 2032     cpa  531.0   622253.0   3383.0  3251.0     36.0   96.0   
       2035     cpa  532.0   622253.0   3474.0  3342.0     36.0   96.0   
       2040     cpa  533.0   622253.0   3554.0  3419.0     39.0   96.0   
       2045     cpa  534.0   622253.0   3593.0  3458.0     39.0   96.0   
       2050     cpa  535.0   622253.0   3640.0  3505.0     39.0   96.0   

                  hh   hh_sf    hh_mf  ...  milestocoast          acres  \
cpa    year                            ...                                
1442.0 2026  24968.0  7570.0  17398.0  ...     1941.9361    2684.380125   
       2029  26460.0  7619.0  18841.0  ...     1941.9361    2684.380125   
       2030  26966.0  7629.0  19337.0  ...     1941.9361    2684.380125   
       2032  27266.0  7632.0  19634.0  ...     1941.9361    2684.380125   
       2035  27637.0  7632.0  20005.0  ...     1941.9361    2684.380125   
...              ...     ...      ...  ...           ...            ...   
1906.0 2032   3236.0  3109.0     36.0  ...     3096.6867  107365.932568   
       2035   3308.0  3181.0     36.0  ...     3096.6867  107365.932568   
       2040   3370.0  3242.0     37.0  ...     3096.6867  107365.932568   
       2045   3415.0  3286.0     38.0  ...     3096.6867  107365.932568   
       2050   3456.0  3326.0     39.0  ...     3096.6867  107365.932568   

             effective_acres     land_acres  MicroAccessTime  remoteAVParking  \
cpa    year                                                                     
1442.0 2026      2640.743833    2684.380125           4600.0              2.0   
       2029      2640.743833    2684.380125           4600.0              2.0   
       2030      2640.743833    2684.380125           4600.0              2.0   
       2032      2640.743833    2684.380125           4600.0              2.0   
       2035      2640.743833    2684.380125           4600.0              2.0   
...                      ...            ...              ...              ...   
1906.0 2032     51550.884299  106651.799938          18960.0              0.0   
       2035     51550.884299  106651.799938          18960.0              0.0   
       2040     51550.884299  106651.799938          18960.0              0.0   
       2045     51550.884299  106651.799938          18960.0              0.0   
       2050     51550.884299  106651.799938          18960.0              0.0   

             refueling_stations  vacancy  vacancy_rate       cpa_name  
cpa    year                                                            
1442.0 2026                 9.0   1588.0      0.059798         Uptown  
       2029                 9.0   1764.0      0.062500         Uptown  
       2030                 9.0   1828.0      0.063485         Uptown  
       2032                 9.0   1790.0      0.061605         Uptown  
       2035                 9.0   1886.0      0.063882         Uptown  
...                         ...      ...           ...            ...  
1906.0 2032                 1.0    147.0      0.043453  Jamul-Dulzura  
       2035                 1.0    166.0      0.047784  Jamul-Dulzura  
       2040                 1.0    184.0      0.051773  Jamul-Dulzura  
       2045                 1.0    178.0      0.049541  Jamul-Dulzura  
       2050                 1.0    184.0      0.050549  Jamul-Dulzura  

[697 rows x 111 columns]

In [11]:
# create df of all differences
def full_diff_pivot(df1, df2, geo, series14 = False):
    
    if 'taz' in list(df1.columns) and geo != 'taz':
        df1 = df1.drop('taz', axis=1)
    if 'LUZ' in list(df1.columns):
        df1 = df1.drop('LUZ', axis=1)
        
    if 'taz' in list(df2.columns) and geo != 'taz':
        df2 = df2.drop('taz', axis=1)
    if 'LUZ' in list(df2.columns):
        df2 = df2.drop('LUZ', axis=1)
    
    # attributes matching
    current_headers = common_values(df1, df2)

    correct_headers = pd.read_csv(
        rf"T:/socioec/Current_Projects/SR15/S0/version6/abm_csv/ForQApurposes/mgra15_based_input_2026_01.csv", nrows=0).columns

    attributes = [header for header in correct_headers if header in current_headers] + ['vacancy', 'vacancy_rate']
    
    if geo not in attributes:
        attributes = [geo] + attributes
    
    # create diffs
    df1_value_name = df1['year'].iloc[0]
    df2_value_name = df2['year'].iloc[0]
    num_years = abs(int(df2_value_name) - int(df1_value_name))
    
    if series14 == 'df2':
        df2_value_name = df2_value_name + ' SR14'
    elif series14 == 'df1':
        df1_value_name = df1_value_name + ' SR14'
    
    df1_pivot = df1[attributes].melt(id_vars=[geo],
                                               var_name = 'attribute',
                                               value_name = df1_value_name)
    df2_pivot = df2[attributes].melt(id_vars=[geo],
                                               var_name = 'attribute',
                                               value_name = df2_value_name)
    full_diff_df =  df1_pivot.merge(df2_pivot,
                                    how='outer',
                                    on=[geo, 'attribute'])
    full_diff_df['diff'] = full_diff_df[df1_value_name] - full_diff_df[df2_value_name]
    full_diff_df['pct diff'] = full_diff_df['diff']/full_diff_df[df2_value_name]
    
    # create CAGRs
    if num_years != 0:
        
        full_diff_df['CAGR'] = ((full_diff_df[df1_value_name]/full_diff_df[df2_value_name])**(1/num_years))-1
        
        # add in thresholds column
        threshold_dict = {'mgra': (350, .03), 'cpa': (500, .03), 'luz': (700, .03)}
        if geo in threshold_dict.keys():
            full_diff_df['threshold_passed'] = (full_diff_df['diff'] < threshold_dict[geo][0]) | (full_diff_df['CAGR'] < threshold_dict[geo][1])
    
    # merging the dataframes together
    full_diff_df = full_diff_df.merge(
        full_diff_df.groupby(['attribute']).sum().reset_index()[['attribute', df1_value_name, df2_value_name]],
        how='outer',
        on=['attribute'],
        suffixes=('',' Share'))
    full_diff_df[df1_value_name + ' Share'] = (full_diff_df[df1_value_name]/full_diff_df[df1_value_name + ' Share'])
    full_diff_df[df2_value_name + ' Share'] = (full_diff_df[df2_value_name]/full_diff_df[df2_value_name + ' Share'])
    full_diff_df['Geographic Share Change'] = full_diff_df[df1_value_name + ' Share'] - full_diff_df[df2_value_name + ' Share']
    
    full_diff_df.insert(0, 'geotype', geo)
    full_diff_df.insert(1, 'geozone', full_diff_df[geo])
    
    return full_diff_df

In [12]:
geo='cpa'
full_diff_pivot(geo_dfs[geo].reset_index()[geo_dfs[geo].reset_index()['year'] == '2026'],
                   geo_dfs[geo].reset_index()[geo_dfs[geo].reset_index()['year'] == '2022'],
                   geo
                   )

,geotype,geozone,cpa,attribute,2026,2022,diff,pct diff,CAGR,threshold_passed,2026 Share,2022 Share,Geographic Share Change
0,cpa,0,0,pop,1.399803e+06,1.399346e+06,457.000000,0.000327,0.000082,True,0.423895,0.425682,-0.001786
1,cpa,1401,1401,pop,4.060000e+02,4.130000e+02,-7.000000,-0.016949,-0.004264,True,0.000123,0.000126,-0.000003
2,cpa,1402,1402,pop,3.861000e+03,3.885000e+03,-24.000000,-0.006178,-0.001548,True,0.001169,0.001182,-0.000013
3,cpa,1403,1403,pop,1.483500e+04,1.229700e+04,2538.000000,0.206392,0.048026,False,0.004492,0.003741,0.000752
4,cpa,1404,1404,pop,6.385800e+04,4.789600e+04,15962.000000,0.333264,0.074556,False,0.019338,0.014570,0.004768
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5363,cpa,1953,1953,vacancy_rate,5.160399e-02,4.935834e-02,0.002246,0.045497,0.011185,True,0.005142,0.004774,0.000367
5364,cpa,1954,1954,vacancy_rate,8.363060e-02,8.270577e-02,0.000925,0.011182,0.002784,True,0.008332,0.008000,0.000332
5365,cpa,1955,1955,vacancy_rate,9.379128e-02,9.247028e-02,0.001321,0.014286,0.003552,True,0.009345,0.008945,0.000400
5366,cpa,1998,1998,vacancy_rate,3.472222e-03,3.472222e-03,0.000000,0.000000,0.000000,True,0.000346,0.000336,0.000010


In [13]:
# adding in to excel files
for i in tqdm(range(1, len(years))):
    writer = pd.ExcelWriter(r'Results/mgra15_based_input_{}_{}_minus_SR14.xlsx'.format(version, years[i]), engine='xlsxwriter')
    for geo in tqdm(geo_dfs.keys()):
        df = full_diff_pivot(geo_dfs[geo].reset_index()[geo_dfs[geo].reset_index()['year'] == years[i]],
                       series14_df[geo].reset_index()[series14_df[geo].reset_index()['year'] == years[i]],
                       geo,
                       series14 = 'df2'
                       )

        if geo == 'cpa':
            df = add_cpa_names(df, 15).rename(columns={'cpa_name':'cpa_name_SR15'})
            df = add_cpa_names(df, 14).rename(columns={'cpa_name':'cpa_name_SR14'})

        df.to_excel(writer, sheet_name = '{}'.format(geo), index=False)
    writer.save()
    
    writer = pd.ExcelWriter(r'Results/mgra15_based_input_{}_{}_minus_{}.xlsx'.format(version, years[i], years[i-1]), engine='xlsxwriter')

    for geo in tqdm(geo_dfs.keys()):
        df = full_diff_pivot(geo_dfs[geo].reset_index()[geo_dfs[geo].reset_index()['year'] == years[i]],
                       geo_dfs[geo].reset_index()[geo_dfs[geo].reset_index()['year'] == years[i-1]],
                       geo
                       )

        if geo == 'mgra':
            df.to_csv(r'Results/mgra15_based_input_{}_{}_minus_{}_{}.csv'.format(version, years[i], years[i-1], geo))
        else:
            df.to_excel(writer, sheet_name = '{}'.format(geo), index=False)
    writer.save()

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:19<00:00, 13.22s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:18<00:00, 13.08s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:25<00:00, 14.21s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:16<00:00, 12.75s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:23<00:00, 13.94s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:44<00:00, 17.38s/it]

100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [18:01<00:00, 180.20s/it]


### Diff columns for hhi and emp

In [14]:
# employment
pattern = re.compile("^emp_")
emp_columns = [col for col in geo_dfs['mgra'].columns if pattern.match(col)]

# household income
pattern = re.compile("i\d+")
household_income_columns = [col for col in geo_dfs['mgra'].columns if pattern.match(col)]

In [15]:
def diff_columns(df, geo, columns, col_var_name, value_name, first_year, second_year):
    def column_pct(df, geo, columns, year, col_var_name):
        pct_df = df[df['year'] == year][[geo] + columns].set_index(geo)
        return (pct_df.T/pct_df.sum(axis=1)).T.reset_index().melt(id_vars = [geo], var_name = col_var_name, value_name=value_name)

    df_values_1 = df[df['year']==first_year][[geo]+columns].melt(id_vars = [geo], var_name = col_var_name, value_name=first_year)
    df1 = column_pct(df, geo, columns, first_year, col_var_name)
    
    df_values_2 = df[df['year']==second_year][[geo]+columns].melt(id_vars = [geo], var_name = col_var_name, value_name=second_year)
    df2 = column_pct(df, geo, columns, second_year, col_var_name)

    output = df_values_1.merge(df_values_2, how='outer', on=[geo, col_var_name])
    output['diff'] = output[first_year] - output[second_year]
    output['pct_diff'] = output['diff']/output[second_year]
    
    num_years = int(first_year) - int(second_year)
    output['CAGR'] = ((output[first_year]/output[second_year])**(1/num_years))-1
    
    output = output.merge(df1, how='outer', on=[geo, col_var_name])
    output = output.merge(df2, how='outer', on=[geo, col_var_name], suffixes=(' '+first_year, ' '+second_year))
    output[value_name + ' Change'] = output[value_name + ' ' + first_year] - output[value_name + ' ' + second_year]
    
    output.insert(0, 'geotype', geo)
    output.insert(1, 'geozone', output[geo])
    return output

In [16]:
for i in tqdm(range(1, len(years))):
    writer = pd.ExcelWriter(r'Results/houeshold_income_share_diffs_{}_{}_minus_{}.xlsx'.format(version, years[i], years[i-1]), engine='xlsxwriter')
    for geo in tqdm(geo_dfs.keys()):
        diff = diff_columns(geo_dfs[geo].reset_index(), geo, household_income_columns, 'hhi', 'Income Share', years[i], years[i-1])
        diff.to_excel(writer, sheet_name = '{}'.format(geo), index=False)
    writer.save()

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [05:45<00:00, 57.52s/it]


In [17]:
for i in tqdm(range(1, len(years))):
    writer = pd.ExcelWriter(r'Results/sector_share_diffs_{}_{}_minus_{}.xlsx'.format(version, years[i], years[i-1]), engine='xlsxwriter')
    for geo in tqdm(geo_dfs.keys()):
        diff = diff_columns(geo_dfs[geo].reset_index(), geo, household_income_columns, 'hhi', 'Sector Share', years[i], years[i-1])
        diff.to_excel(writer, sheet_name = '{}'.format(geo), index=False)
    writer.save()

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [06:11<00:00, 62.00s/it]


### Consolidated CGAR

In [18]:
geo_dfs['mgra'].reset_index()\
                                    .sort_values(['mgra', 'year'])\
                                    .set_index(['mgra', 'year'])\
                                    .select_dtypes(include='number')

scenario  increment   taz  luz    zip  pseudomsa  district27  pop  \
mgra  year                                                                      
1     2022         0       2022  3010   10  92105          2           9  440   
      2026         0       2026  3010   10  92105          2           9  414   
      2029         0       2029  3010   10  92105          2           9  409   
      2032         0       2032  3010   10  92105          2           9  406   
      2035         0       2035  3010   10  92105          2           9  401   
...              ...        ...   ...  ...    ...        ...         ...  ...   
24321 2029         0       2029  3039  200  92026          7           3   57   
      2032         0       2032  3039  200  92026          7           3   56   
      2035         0       2035  3039  200  92026          7           3   56   
      2040         0       2040  3039  200  92026          7           3   55   
      2050         0       2050  3039  200  92026          7           3   54   

            hhp  gq  ...  redev_emp_res  redev_emp_emp  infill_sf  infill_mf  \
mgra  year           ...                                                       
1     2022  440   0  ...            0.0            0.0        0.0        0.0   
      2026  414   0  ...            0.0            0.0        0.0        0.0   
      2029  409   0  ...            0.0            0.0        0.0        0.0   
      2032  406   0  ...            0.0            0.0        0.0        0.0   
      2035  401   0  ...            0.0            0.0        0.0        0.0   
...         ...  ..  ...            ...            ...        ...        ...   
24321 2029   57   0  ...            0.0            0.0        0.0        0.0   
      2032   56   0  ...            0.0            0.0        0.0        0.0   
      2035   56   0  ...            0.0            0.0        0.0        0.0   
      2040   55   0  ...            0.0            0.0        0.0        0.0   
      2050   54   0  ...            0.0            0.0        0.0        0.0   

            infill_emp        dev      vac  unusable  vacancy  vacancy_rate  
mgra  year                                                                   
1     2022         0.0  18.837695  0.00000       0.0        2      0.011364  
      2026         0.0  18.837695  0.00000       0.0        8      0.045455  
      2029         0.0  18.837695  0.00000       0.0        8      0.045455  
      2032         0.0  18.837695  0.00000       0.0        8      0.045455  
      2035         0.0  18.837695  0.00000       0.0        8      0.045455  
...                ...        ...      ...       ...      ...           ...  
24321 2029         0.0   7.849918  0.43073       0.0        2      0.086957  
      2032         0.0   7.849918  0.43073       0.0        2      0.086957  
      2035         0.0   7.849918  0.43073       0.0        2      0.086957  
      2040         0.0   7.849918  0.43073       0.0        2      0.086957  
      2050         0.0   7.849918  0.43073       0.0        2      0.086957  

[170247 rows x 126 columns]

In [19]:
writer = pd.ExcelWriter(r'Data/mgra15_based_input_consolidated_CGAR_{}.xlsx'.format(version), engine='xlsxwriter')

for geo in tqdm(geo_dfs.keys()):
    consolidated_cgar = geo_dfs[geo].reset_index()\
                                        .sort_values([geo, 'year'])\
                                        .set_index([geo, 'year'])\
                                        .select_dtypes(include='number')

    year_diffs = consolidated_cgar.reset_index()
    year_diffs = year_diffs.assign(year_diff = year_diffs['year'].astype(int) - year_diffs['year'].astype(int).shift(1))
    year_diffs = year_diffs.assign(year = year_diffs['year'].astype(str)).set_index([geo, 'year'])

    cgar = consolidated_cgar.div(consolidated_cgar.shift(1))
    cgar = ((cgar.T.pow(1/year_diffs['year_diff']).T)-1).reset_index()
    
    cgar[cgar['year']!='2022'].sort_values([geo, 'year']).to_excel(writer, sheet_name = '{}'.format(geo), index=False)
writer.save()

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [03:28<00:00, 34.81s/it]


### Negative values (Region, Jurisdiction, CPA, LUZ, MGRA)

In [20]:
for geo, df in geo_dfs.items():
    df = df.reset_index()[df.reset_index()['year'] != '2022']
    print('------------------------------------------------------------------')
    print(geo, df[(df.select_dtypes(include='number') < 0).any(axis=1)].shape)
    print('******************************************************************')
    display(df.select_dtypes(include='number').loc[:,(df.select_dtypes(include='number') < 0).any()])

------------------------------------------------------------------
mgra (0, 129)
******************************************************************


""
24321
24322
24323
24324
24325
...
170242
170243
170244
170245


------------------------------------------------------------------
region (0, 129)
******************************************************************


""
1
2
3
4
5
6


------------------------------------------------------------------
jurisdiction (0, 129)
******************************************************************


...

""
1
2
3
4
5
...
611
612
613
614


------------------------------------------------------------------
luz (0, 128)
******************************************************************


""
1
2
3
4
5
...
1710
1711
1712
1713


------------------------------------------------------------------
taz (0, 128)
******************************************************************


""
1
2
3
4
5
...
34540
34541
34542
34543


### Zero Values (Region, Jurisdiction, CPA, LUZ) 

In [21]:
for geo, df in geo_dfs.items():
    df = df.reset_index()[df.reset_index()['year'] != '2022'].loc[:, df.reset_index().columns != 'scenario']
    print('------------------------------------------------------------------')
    print(geo, df[(df.select_dtypes(include='number') == 0).any(axis=1)].shape)
    print('******************************************************************')
    display(df.select_dtypes(include='number').loc[:,(df.select_dtypes(include='number') == 0).any()])

------------------------------------------------------------------
mgra (145926, 128)
******************************************************************


,pop,hhp,gq,gq_mil,gq_civ,gq_civ_college,gq_civ_other,er,hhs,hs,...,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate
24321,0,0,0,0,0,0,0,0,0.000000,0,...,0.0,0.0,0.000000,0.0,0.0,10.994980,0.000000,0.0,0,NaN
24322,0,0,0,0,0,0,0,0,0.000000,25,...,0.0,0.0,0.000000,0.0,0.0,2.287811,0.000000,0.0,25,1.000000
24323,208,208,0,0,0,0,0,74,2.929577,71,...,0.0,0.0,3.202705,0.0,0.0,16.020178,5.274488,0.0,0,0.000000
24324,205,205,0,0,0,0,0,71,2.928571,72,...,0.0,0.0,0.000000,0.0,0.0,8.799739,0.000000,0.0,2,0.027778
24325,490,490,0,0,0,0,0,210,2.768362,189,...,0.0,0.0,0.000000,0.0,0.0,21.119591,0.084978,0.0,12,0.063492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170242,13,13,0,0,0,0,0,7,2.600000,9,...,0.0,0.0,0.000000,0.0,0.0,612.689264,0.000000,0.0,4,0.444444
170243,13,13,0,0,0,0,0,7,2.600000,10,...,0.0,0.0,0.000000,0.0,0.0,379.892975,0.000000,0.0,5,0.500000
170244,112,112,0,0,0,0,0,51,2.434783,47,...,0.0,0.0,0.000000,0.0,0.0,227.739678,0.000000,0.0,1,0.021277
170245,31,31,0,0,0,0,0,14,2.384615,18,...,0.0,0.0,0.000000,0.0,0.0,92.737743,0.000000,0.0,5,0.277778


------------------------------------------------------------------
region (6, 128)
******************************************************************


,redev_mh_sf,redev_mh_mf,redev_mh_emp
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0
5,0.0,0.0,0.0
6,0.0,0.0,0.0


------------------------------------------------------------------
jurisdiction (114, 128)
******************************************************************


...

,cpa,pop,hhp,gq,gq_mil,gq_civ,gq_civ_college,gq_civ_other,er,hs,...,redev_ag_roads,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy
1,0,1399803,1373636,26167,9141,17026,1751,15275,664891,518602,...,1.766242,720.859024,1656.061357,29.169509,506.366823,4320.317121,180853.203741,11394.838346,31874.072908,29158
2,0,1417019,1390503,26516,9221,17295,1775,15520,673445,532053,...,1.766242,589.549752,1597.243749,22.780979,461.758200,4171.256838,182341.525777,9906.516309,31874.072908,31607
3,0,1451434,1424008,27426,9361,18065,1945,16120,689540,549073,...,1.766242,546.926942,1517.895996,7.672304,407.280975,4012.085057,183685.580225,8561.040730,31874.072908,32627
4,0,1466553,1438544,28009,9501,18508,2060,16448,697726,559728,...,1.766242,528.029305,1463.489250,2.510639,342.874423,3688.324460,184656.797300,7590.854737,31874.072908,33390
5,0,1476557,1448813,27744,9428,18316,2008,16308,705303,574344,...,1.766242,268.440291,1296.733892,0.000000,309.314874,3280.558621,185544.418513,6708.957489,31874.072908,35154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,1999,2539,2539,0,0,0,0,0,967,743,...,0.000000,46.655787,64.510573,2.146079,0.000000,0.000000,366.552780,8.717923,0.000000,40
612,1999,2537,2537,0,0,0,0,0,970,751,...,0.000000,46.576316,64.306363,2.146079,0.000000,0.000000,366.927985,8.342718,0.000000,40
613,1999,2577,2577,0,0,0,0,0,990,771,...,0.000000,46.496845,63.871183,2.056487,0.000000,0.000000,367.817233,7.453470,0.000000,40
614,1999,2525,2525,0,0,0,0,0,981,776,...,0.000000,46.496845,62.786514,2.056487,0.000000,0.000000,368.002352,7.268351,0.000000,42


------------------------------------------------------------------
luz (1470, 127)
******************************************************************


,pop,hhp,gq,gq_mil,gq_civ,gq_civ_college,gq_civ_other,er,hs,hs_sf,...,redev_ag_roads,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,vac,unusable,vacancy,vacancy_rate
1,7515,7474,41,0,41,0,41,4924,4045,2385,...,0.0,0.000000,17.192360,0.000000,0.000000,6.912115,13.729838,59.487941,217,0.053646
2,7596,7555,41,0,41,0,41,4963,4131,2361,...,0.0,0.000000,17.192360,0.000000,0.000000,6.912115,13.576919,59.487941,236,0.057129
3,7621,7580,41,0,41,0,41,4975,4174,2298,...,0.0,0.000000,17.192360,0.000000,0.000000,6.912115,13.390503,59.487941,239,0.057259
4,7735,7694,41,0,41,0,41,5044,4260,2278,...,0.0,0.000000,17.192360,0.000000,0.000000,6.912115,13.132746,59.487941,242,0.056808
5,7196,7155,41,0,41,0,41,4689,4295,2253,...,0.0,0.000000,17.192360,0.000000,0.000000,6.912115,13.091438,59.487941,510,0.118743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710,10707,10637,70,0,70,0,70,4995,3875,1140,...,0.0,4.691822,93.587226,77.829521,27.032324,8.162603,19.036380,0.000000,56,0.014452
1711,10646,10576,70,0,70,0,70,4975,3890,1153,...,0.0,4.691822,92.369799,74.730239,26.902801,8.162603,18.724097,0.000000,56,0.014396
1712,10668,10597,71,0,71,0,71,4996,3932,1179,...,0.0,4.491585,87.975826,68.224587,25.996143,8.162603,17.400998,0.000000,55,0.013988
1713,11134,11064,70,0,70,0,70,5245,4192,1411,...,0.0,3.721173,73.319766,0.000000,24.441871,8.162603,10.712536,0.000000,67,0.015983


------------------------------------------------------------------
taz (29610, 127)
******************************************************************


,pop,hhp,gq,gq_mil,gq_civ,gq_civ_college,gq_civ_other,er,hs,hs_sf,...,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable,vacancy,vacancy_rate
1,4591,2831,1760,1760,0,0,0,1211,993,993,...,0.0,0.0,0.0,0.0,0.0,4889.004243,0.00000,0.0,66,0.066465
2,4565,2796,1769,1769,0,0,0,1197,993,993,...,0.0,0.0,0.0,0.0,0.0,4889.004243,0.00000,0.0,67,0.067472
3,4550,2767,1783,1783,0,0,0,1188,993,993,...,0.0,0.0,0.0,0.0,0.0,4889.004243,0.00000,0.0,67,0.067472
4,4536,2740,1796,1796,0,0,0,1180,993,993,...,0.0,0.0,0.0,0.0,0.0,4889.004243,0.00000,0.0,67,0.067472
5,4472,2682,1790,1790,0,0,0,1163,993,993,...,0.0,0.0,0.0,0.0,0.0,4889.004243,0.00000,0.0,68,0.068479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34540,7,7,0,0,0,0,0,6,8,8,...,0.0,0.0,0.0,0.0,0.0,960.073851,270.55383,0.0,5,0.625000
34541,7,7,0,0,0,0,0,6,8,8,...,0.0,0.0,0.0,0.0,0.0,960.073851,270.55383,0.0,5,0.625000
34542,7,7,0,0,0,0,0,6,8,8,...,0.0,0.0,0.0,0.0,0.0,960.073851,270.55383,0.0,5,0.625000
34543,7,7,0,0,0,0,0,6,8,8,...,0.0,0.0,0.0,0.0,0.0,960.073851,270.55383,0.0,5,0.625000


### Previous Version Comparison

In [4]:
query = """
Select *
from [sr15_staging].[2023_11_29].[mgrabase]
"""

prev_mgra_data = pd.read_sql(query, con=engine.connect())
prev_mgra_data

,scenario,increment,mgra,taz,luz,zip,pseudomsa,district27,pop,hhp,...,redev_ag_schools,redev_ag_roads,redev_emp_res,redev_emp_emp,infill_sf,infill_mf,infill_emp,dev,vac,unusable
0,0,2035,365,2469,92,91932,4,11,200,200,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,17.048736,0.0,0.0
1,0,2035,665,3364,11,92115,2,9,57,57,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,4.689694,0.0,0.0
2,0,2035,1535,2383,68,92126,3,4,2865,0,...,0.0,0.0,0.0,6.537975,0.0,0.0,0.0,122.979187,0.0,0.0
3,0,2035,4943,4198,105,91941,5,13,53,53,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,22.194187,0.0,0.0
4,0,2035,4945,3328,74,92120,3,6,0,0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,13.391927,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170242,0,2050,22878,4780,114,91901,5,14,431,431,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,31.309829,0.0,0.0
170243,0,2050,22893,4793,114,91901,5,14,637,534,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,13.250893,0.0,0.0
170244,0,2050,22927,4769,114,91901,5,14,0,0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,41.721484,0.0,0.0
170245,0,2050,22929,4769,114,91901,5,14,0,0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,21.000025,0.0,0.0


In [12]:
ver_comp = (prev_mgra_data.sort_values(['increment', 'mgra']).set_index(['increment', 'mgra'])!=\
full_mgra_data.sort_values(['increment', 'mgra']).set_index(['increment', 'mgra']))
dict(ver_comp.sum())

{'scenario': 0,
 'taz': 0,
 'luz': 0,
 'zip': 0,
 'pseudomsa': 0,
 'district27': 0,
 'pop': 77180,
 'hhp': 76128,
 'gq': 2889,
 'gq_mil': 132,
 'gq_civ': 2757,
 'gq_civ_college': 163,
 'gq_civ_other': 2713,
 'er': 43166,
 'hhs': 76251,
 'hs': 10872,
 'hs_sf': 6494,
 'hs_mf': 7465,
 'hs_mh': 0,
 'hh': 26924,
 'hh_sf': 11856,
 'hh_mf': 18789,
 'hh_mh': 0,
 'emp_tot': 7367,
 'emp_tot_ws_only': 5217,
 'emp_civ': 7367,
 'emp_civ_ws_only': 5217,
 'emp_mil': 0,
 'emp_gov': 5839,
 'emp_ag_min': 3354,
 'emp_bus_svcs': 27180,
 'emp_fin_res_mgm': 17956,
 'emp_educ': 10623,
 'emp_hlth': 30210,
 'emp_ret': 16429,
 'emp_trn_wrh': 8241,
 'emp_con': 18460,
 'emp_utl': 3437,
 'emp_mnf': 10008,
 'emp_whl': 10597,
 'emp_ent': 8111,
 'emp_accm': 5060,
 'emp_food': 15453,
 'emp_oth': 18982,
 'emp_non_ws_wfh': 1411,
 'emp_non_ws_oth': 884,
 'emp_indus_lu': 1187,
 'emp_comm_lu': 4477,
 'emp_office_lu': 1020,
 'emp_other_lu': 2749,
 'i1': 76160,
 'i2': 81054,
 'i3': 83645,
 'i4': 83505,
 'i5': 82939,
 'i6': 8

## PASEF rollup

In [3]:
server = 'ddamwsql16'
database = 'sr15_dev'
driver = 'ODBC Driver 17 for SQL Server'

engine = sql.create_engine('mssql+pyodbc://@{}/{}?trusted_connection=yes&driver={}'.format(server, database, driver))

query = """
Select *
from [sr15_staging].[2023_12_05].[pop_ase_mgra]
"""

pasef = pd.read_sql(query, con=engine.connect())
pasef['key'] = pasef['ethnicity_id'].astype(str)+'_'+pasef['sex_id'].astype(str)+'_'+pasef['age_group_id'].astype(str)
pasef

,scenario,increment,mgra,ethnicity_id,sex_id,age_group_id,pop,hhp,gq,key
0,0,2022,3730,2,2,7,0,0,0,2_2_7
1,0,2022,6041,2,2,8,0,0,0,2_2_8
2,0,2022,7421,4,2,10,0,0,0,4_2_10
3,0,2022,8767,5,1,5,0,0,0,5_1_5
4,0,2022,13924,2,2,4,0,0,0,2_2_4
...,...,...,...,...,...,...,...,...,...,...
54479035,0,2050,24321,8,2,16,0,0,0,8_2_16
54479036,0,2050,24321,8,2,17,0,0,0,8_2_17
54479037,0,2050,24321,8,2,18,0,0,0,8_2_18
54479038,0,2050,24321,8,2,19,0,0,0,8_2_19


In [4]:
def create_pasef_denorm_table(mgra_denorm_path, geo_level, version, series):

    if type(mgra_denorm_path) != str:
        df_1 = mgra_denorm_path
    else:
        df_1 = import_mgra_based_data(path = mgra_denorm_path)

    df_2 = download_mgra_denorm_data(geo_level=geo_level, series=series)
    
    df_3 = pd.merge(df_2, df_1, how='left')

    if geo_level != 'mgra':
        df_3 = df_3.drop('mgra', axis=1)

    if 'year' in list(df_3.columns):
        df_3 = df_3.groupby([geo_level, 'year', 'key']).sum()
    else:
        df_3 = df_3.groupby([geo_level, 'key']).sum()

    return df_3

In [5]:
pasef_dfs = {}
pasef_dfs['mgra'] = pasef.rename(columns={'increment':'year'})
for geo in tqdm(['region', 'jurisdiction', 'cpa', 'luz']):
    pasef_dfs[geo] = create_pasef_denorm_table(pasef_dfs['mgra'], 
                         geo_level=geo,
                         version='SR15',
                         series=15)
    if geo == 'cpa':
        pasef_dfs[geo] = add_cpa_names(pasef_dfs[geo], 15).set_index(['cpa', 'year'])
    pasef_dfs[geo].insert(0, 'geotype', geo)
    display(pasef_dfs[geo])
pasef_dfs['luz']

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

geotype  scenario  ethnicity_id  sex_id  age_group_id  pop  \
luz year key                                                                 
1   2022 1_1_1      luz         0           148     148           148   48   
         1_1_10     luz         0           148     148          1480   27   
         1_1_11     luz         0           148     148          1628   52   
         1_1_12     luz         0           148     148          1776   21   
         1_1_13     luz         0           148     148          1924   19   
...                 ...       ...           ...     ...           ...  ...   
245 2050 8_2_5      luz         0           312      78           195    6   
         8_2_6      luz         0           312      78           234   17   
         8_2_7      luz         0           312      78           273   21   
         8_2_8      luz         0           312      78           312    2   
         8_2_9      luz         0           312      78           351    2   

                 hhp  gq  
luz year key              
1   2022 1_1_1    48   0  
         1_1_10   27   0  
         1_1_11   52   0  
         1_1_12   21   0  
         1_1_13   19   0  
...              ...  ..  
245 2050 8_2_5     6   0  
         8_2_6    17   0  
         8_2_7    21   0  
         8_2_8     2   0  
         8_2_9     2   0  

[548800 rows x 8 columns]

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [02:46<00:00, 166.89s/it]


geotype  scenario  ethnicity_id  sex_id  age_group_id  pop  \
luz year key                                                                 
1   2022 1_1_1      luz         0           148     148           148   48   
         1_1_10     luz         0           148     148          1480   27   
         1_1_11     luz         0           148     148          1628   52   
         1_1_12     luz         0           148     148          1776   21   
         1_1_13     luz         0           148     148          1924   19   
...                 ...       ...           ...     ...           ...  ...   
245 2050 8_2_5      luz         0           312      78           195    6   
         8_2_6      luz         0           312      78           234   17   
         8_2_7      luz         0           312      78           273   21   
         8_2_8      luz         0           312      78           312    2   
         8_2_9      luz         0           312      78           351    2   

                 hhp  gq  
luz year key              
1   2022 1_1_1    48   0  
         1_1_10   27   0  
         1_1_11   52   0  
         1_1_12   21   0  
         1_1_13   19   0  
...              ...  ..  
245 2050 8_2_5     6   0  
         8_2_6    17   0  
         8_2_7    21   0  
         8_2_8     2   0  
         8_2_9     2   0  

[548800 rows x 8 columns]

In [6]:
# consolidated files:
writer = pd.ExcelWriter(r'Data/pasef_luz_{}.xlsx'.format(version), engine='xlsxwriter')
for geo, df in tqdm(pasef_dfs.items()):
    if geo != 'mgra':
        display(df)
        df.reset_index().sort_values([geo, 'key', 'year']).to_excel(writer, sheet_name = '{}'.format(geo), index=False)
writer.save()

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

geotype  scenario  ethnicity_id  sex_id  age_group_id  pop  \
luz year key                                                                 
1   2022 1_1_1      luz         0           148     148           148   48   
         1_1_10     luz         0           148     148          1480   27   
         1_1_11     luz         0           148     148          1628   52   
         1_1_12     luz         0           148     148          1776   21   
         1_1_13     luz         0           148     148          1924   19   
...                 ...       ...           ...     ...           ...  ...   
245 2050 8_2_5      luz         0           312      78           195    6   
         8_2_6      luz         0           312      78           234   17   
         8_2_7      luz         0           312      78           273   21   
         8_2_8      luz         0           312      78           312    2   
         8_2_9      luz         0           312      78           351    2   

                 hhp  gq  
luz year key              
1   2022 1_1_1    48   0  
         1_1_10   27   0  
         1_1_11   52   0  
         1_1_12   21   0  
         1_1_13   19   0  
...              ...  ..  
245 2050 8_2_5     6   0  
         8_2_6    17   0  
         8_2_7    21   0  
         8_2_8     2   0  
         8_2_9     2   0  

[548800 rows x 8 columns]

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:46<00:00, 23.45s/it]


In [7]:
server = 'ddamwsql16'
database = 'sr15_dev'
driver = 'ODBC Driver 17 for SQL Server'

engine = sql.create_engine('mssql+pyodbc://@{}/{}?trusted_connection=yes&driver={}'.format(server, database, driver))

ase_df = {}
ase_df['ethnicity_id'] = {}
ase_df['sex_id'] = {}
ase_df['age_group_id'] = {}

for ase_id in ['ethnicity_id', 'sex_id', 'age_group_id']:
    query = """
    Select increment, mgra, {ase},
    SUM(pop) as pop,
    SUM(hhp) as hhp,
    SUM(gq) as gq
    from [sr15_staging].[2023_12_05].[pop_ase_mgra] as pasef
    GROUP BY increment, mgra, {ase}
    """.format(ase = ase_id)

    pasef_ase = pd.read_sql(query, con=engine.connect())
    pasef_ase = pasef_ase.rename(columns={'increment':'year'})
    ase_df[ase_id]['mgra'] = pasef_ase
    
    for geo in tqdm(['region', 'jurisdiction', 'cpa', 'luz'])
        denorm = download_mgra_denorm_data(geo_level=geo, series=15)
        ase_df[ase_id][geo] = pd.merge(denorm, ase_df[ase_id]['mgra'], how='left')\
                                .drop('mgra', axis=1)\
                                .groupby([geo, 'year', ase_id])\
                                .sum()
        if geo == 'cpa':
            ase_df[ase_id][geo] = add_cpa_names(ase_df[ase_id][geo], 15).set_index(['cpa', 'year'])
        ase_df[ase_id][geo].insert(0, 'geotype', geo)
        display(ase_df[ase_id][geo])

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

geotype   pop   hhp  gq
luz year ethnicity_id                        
1   2022 1                luz   951   942   9
         2                luz  5271  5253  18
         3                luz   180   170  10
         4                luz     8     8   0
         5                luz   448   445   3
...                       ...   ...   ...  ..
245 2050 4                luz    56    56   0
         5                luz   763   752  11
         6                luz   128   128   0
         7                luz    66    66   0
         8                luz   469   467   2

[13720 rows x 4 columns]

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

geotype   pop   hhp  gq
luz year sex_id                        
1   2022 1          luz  3677  3661  16
         2          luz  3556  3530  26
    2026 1          luz  3814  3794  20
         2          luz  3701  3680  21
    2029 1          luz  3898  3879  19
...                 ...   ...   ...  ..
245 2035 2          luz  4658  4619  39
    2040 1          luz  6268  6236  32
         2          luz  4866  4828  38
    2050 1          luz  6198  6170  28
         2          luz  4713  4679  34

[3430 rows x 4 columns]

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

geotype  pop  hhp  gq
luz year age_group_id                      
1   2022 1                luz  213  213   0
         2                luz  404  404   0
         3                luz  282  281   1
         4                luz  126  126   0
         5                luz   10   10   0
...                       ...  ...  ...  ..
245 2050 16               luz  434  431   3
         17               luz  389  377  12
         18               luz  275  265  10
         19               luz  276  273   3
         20               luz  606  600   6

[34300 rows x 4 columns]

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.22s/it]


In [9]:
ase_df['sex_id']['luz']

geotype   pop   hhp  gq
luz year sex_id                        
1   2022 1          luz  3677  3661  16
         2          luz  3556  3530  26
    2026 1          luz  3814  3794  20
         2          luz  3701  3680  21
    2029 1          luz  3898  3879  19
...                 ...   ...   ...  ..
245 2035 2          luz  4658  4619  39
    2040 1          luz  6268  6236  32
         2          luz  4866  4828  38
    2050 1          luz  6198  6170  28
         2          luz  4713  4679  34

[3430 rows x 4 columns]

In [10]:
for ase_id in ['ethnicity_id', 'sex_id', 'age_group_id']:
    # consolidated files:
    writer = pd.ExcelWriter(r'Data/pasef_{}_luz_{}.xlsx'.format(ase_id, version), engine='xlsxwriter')
    for geo, df in tqdm(ase_df[ase_id].items()):
        if geo != 'mgra':
            display(df)
            df.reset_index().sort_values([geo, 'year', ase_id]).to_excel(writer, sheet_name = '{}'.format(geo), index=False)
    writer.save()

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

geotype   pop   hhp  gq
luz year ethnicity_id                        
1   2022 1                luz   951   942   9
         2                luz  5271  5253  18
         3                luz   180   170  10
         4                luz     8     8   0
         5                luz   448   445   3
...                       ...   ...   ...  ..
245 2050 4                luz    56    56   0
         5                luz   763   752  11
         6                luz   128   128   0
         7                luz    66    66   0
         8                luz   469   467   2

[13720 rows x 4 columns]

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

geotype   pop   hhp  gq
luz year sex_id                        
1   2022 1          luz  3677  3661  16
         2          luz  3556  3530  26
    2026 1          luz  3814  3794  20
         2          luz  3701  3680  21
    2029 1          luz  3898  3879  19
...                 ...   ...   ...  ..
245 2035 2          luz  4658  4619  39
    2040 1          luz  6268  6236  32
         2          luz  4866  4828  38
    2050 1          luz  6198  6170  28
         2          luz  4713  4679  34

[3430 rows x 4 columns]

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

geotype  pop  hhp  gq
luz year age_group_id                      
1   2022 1                luz  213  213   0
         2                luz  404  404   0
         3                luz  282  281   1
         4                luz  126  126   0
         5                luz   10   10   0
...                       ...  ...  ...  ..
245 2050 16               luz  434  431   3
         17               luz  389  377  12
         18               luz  275  265  10
         19               luz  276  273   3
         20               luz  606  600   6

[34300 rows x 4 columns]

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.18it/s]


### PASEF Regional Controls Check

In [101]:
query = """
SELECT pasef.increment as year, eth.long_name as race, sex.abbreviation as sex, age.name as age, pop as total_population, hhp, gq as gq_pop
FROM (
select increment, ethnicity_id, sex_id, age_group_id, SUM(pop) as pop, SUM(hhp) as hhp, SUM(gq) as gq
from [sr15_staging].[2023_12_05].[pop_ase_mgra]
group by increment, ethnicity_id, sex_id, age_group_id) AS pasef
LEFT JOIN demographic_warehouse.dim.ethnicity AS eth
ON eth.ethnicity_id = pasef.ethnicity_id
LEFT JOIN demographic_warehouse.dim.sex AS sex
ON sex.sex_id = pasef.sex_id
LEFT JOIN demographic_warehouse.dim.age_group AS age
ON age.age_group_id = pasef.age_group_id
"""

race_dict = {
    'Non-Hispanic, American Indian or Alaska Native': 'AIAN_NH',
    'Non-Hispanic, White': 'White_NH',
    'Non-Hispanic, Other': 'White_NH',
    'Non-Hispanic, Black': 'Black_NH',
    'Non-Hispanic, Hawaiian or Pacific Islander': 'NHPI_NH',
    'Non-Hispanic, Two or More Races': 'Multi-Race NH',
    'Non-Hispanic, Asian': 'Asian_NH'
}

regional_output = pd.read_sql(query, con=engine.connect())
regional = regional_output.copy()
regional['race'] = regional_output['race'].replace(race_dict)

def age_groups(age):
    if age < 5: return 'Under 5'
    elif age < 10: return '5 to 9'
    elif age < 15: return '10 to 14'
    elif age < 18: return '15 to 17'
    elif age < 20: return '18 and 19'
    elif age < 25: return '20 to 24'
    elif age < 30: return '25 to 29'
    elif age < 35: return '30 to 34'
    elif age < 40: return '35 to 39'
    elif age < 45: return '40 to 44'
    elif age < 50: return '45 to 49'
    elif age < 55: return '50 to 54'
    elif age < 60: return '55 to 59'
    elif age < 62: return '60 and 61'
    elif age < 65: return '62 to 64'
    elif age < 70: return '65 to 69'
    elif age < 75: return '70 to 74'
    elif age < 80: return '75 to 79'
    elif age < 85: return '80 to 84'
    else: return '85 and Older'
ccm_output = pd.read_excel(r'C:\Users\jchu\OneDrive - San Diego Association of Governments\Projects\2023\2023-040 CCM Forecast Output (series 15)\Data\Version 5\Household_ characteristics_2020_2060_08312023.xlsx',
                   sheet_name = 'data')
ccm_output['hhp'] = ccm_output['total_population'] - ccm_output['gq_pop']
ccm = ccm_output.copy()

ccm['age'] = ccm['age'].apply(age_groups)
ccm = ccm[ccm['year'].isin([2022, 2026, 2029, 2032, 2035, 2040, 2050])]
ccm = ccm[regional.columns].sort_values(['year', 'race', 'sex', 'age']).groupby(['year', 'race', 'sex', 'age']).sum()
regional = regional.sort_values(['year', 'race', 'sex', 'age']).groupby(['year', 'race', 'sex', 'age']).sum()

display(regional)
display(ccm)

total_population    hhp  gq_pop
year race     sex age                                          
2022 AIAN_NH  F   10 to 14                   367    362       5
                  15 to 17                   182    173       9
                  18 and 19                  127     71      56
                  20 to 24                   491    334     157
                  25 to 29                   600    572      28
...                                          ...    ...     ...
2050 White_NH M   70 to 74                 38153  37256     897
                  75 to 79                 34125  33233     892
                  80 to 84                 28098  27078    1020
                  85 and Older             31125  29420    1705
                  Under 5                  26119  26108      11

[1960 rows x 3 columns]

total_population    hhp  gq_pop
year race     sex age                                          
2022 AIAN_NH  F   10 to 14                   367    362       5
                  15 to 17                   182    173       9
                  18 and 19                  127     71      56
                  20 to 24                   491    334     157
                  25 to 29                   600    572      28
...                                          ...    ...     ...
2050 White_NH M   70 to 74                 38153  37256     897
                  75 to 79                 34125  33233     892
                  80 to 84                 28098  27078    1020
                  85 and Older             31125  29420    1705
                  Under 5                  26119  26108      11

[1960 rows x 3 columns]

In [102]:
pasef_control_check = (ccm != regional)
pasef_control_check = pd.DataFrame(pasef_control_check)
pasef_control_check[(pasef_control_check['total_population'] == True) | (pasef_control_check['hhp'] == True) | (pasef_control_check['gq_pop'] == True)]

,,,,total_population,hhp,gq_pop
year,race,sex,age,,,
